In [6]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F9 '''
def F9(X):
    f = bn.F9()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 395.18 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_9_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_9_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [10]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [11]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(20,30):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-20,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 20
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=248269, Thu Apr 16 18:34:33 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 4.049135404915026e+07 1.0e+00 2.43e+00  2e+00  2e+00 0:00.1
    2     38 4.012244874589280e+07 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [40737199.31680512] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [40750071.58105618] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 4.019288954662453e+07 1.0e+00 2.33e+00  2e+00  2e+00 0:00.3
   30    570 3.663906229781745e+07 1.1e+00 1.89e+00  2e+00  2e+00 0:03.4
   66   1254 3.510508490118027e+07 1.1e+00 1.56e+00  2e+00  2e+00 0:07.5
  100   1900 3.195070009524205e+07 1.2e+00 1.25e+00  1e+00  1e+00 0:11.4
  153   2907 1.802221795117298e+07 1.2e+00 7.75e-01  8e-01  8e-01 0:17.4
  200   3800 5.763927502621904e+06 1.2e+00 4.48e-01  4e-01  4e-01 0:22.8
  269   5111 -3.090104851578176e+05 1.2e+00 2.12e-01  2e-01  2e-01 0:30.9
  300   5700 -1.392950724470824e+06 1.2e+00 1.49e-01  1e-01  1e-01 0:34.3
  389   7391 -2.423345883850276e+06 1.2e+00 5.71e-02  6e-02  6e-02 0:44.3
  400   7600 -2.468076504820578e+06 1.2e+00 5.09e-02  5e-02  5e-02 0:45.6
  500   9500 -2.576241725677133e+06 1.2e+00 1.80e-02  2e-02  2e-02 0:57.0
  600  11400 -2.590746529222071e+06 1.2e+00 6.28e-03  6e-03  6e-03 1:08.2
  700  13300 -2.592443775849305e+06 1.2e+00 2.01e-03  2e-03  2e-03 1:19.9
  800  15200 -2.592627793940477e+06 1.2e+00 

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [40597407.50487186] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=40597407.504872, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=40597407.504872, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

termination on tolfun=1e-11 (Thu Apr 16 18:57:17 2020)
final/bestever f-value = 4.059741e+07 4.059741e+07
incumbent solution: [ 2.22903714 -4.95032768 -2.32455249  4.98944112 -4.97800538  0.23622632
  3.4826546   2.73611769 ...]
std deviations: [2.43707525 2.43740599 2.4375585  2.43747331 2.43741874 2.4369508
 2.43733559 2.43713856 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=169595, Thu Apr 16 18:57:17 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 3.720412932590160e+07 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 3.844538477737780e+07 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


...

    3     57 3.982080616884245e+07 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   29    551 3.859259909234054e+07 1.1e+00 1.94e+00  2e+00  2e+00 0:03.4
   64   1216 3.568941252199429e+07 1.1e+00 1.69e+00  2e+00  2e+00 0:07.4
  100   1900 3.128232468257046e+07 1.2e+00 1.31e+00  1e+00  1e+00 0:11.6
  153   2907 1.829717438816921e+07 1.2e+00 8.17e-01  8e-01  8e-01 0:17.7
  200   3800 9.833203813363396e+06 1.2e+00 5.00e-01  5e-01  5e-01 0:23.0
  271   5149 1.173125427301228e+05 1.2e+00 2.32e-01  2e-01  2e-01 0:31.1
  300   5700 -1.318665932473406e+06 1.2e+00 1.67e-01  2e-01  2e-01 0:34.4
  388   7372 -2.374428980910547e+06 1.2e+00 6.56e-02  6e-02  7e-02 0:44.4
  400   7600 -2.435789386715300e+06 1.2e+00 5.82e-02  6e-02  6e-02 0:45.8
  500   9500 -2.569083113424443e+06 1.2e+00 1.98e-02  2e-02  2e-02 0:57.3
  600  11400 -2.590297996766195e+06 1.2e+00 7.04e-03  7e-03  7e-03 1:09.1
  700  13300 -2.592318988049269e+06 1.2e+00 2.45e-03  2e-03  2e-03 1:20.6
  800  15200 -2.592613810122110e+06 1.2e+00 8

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [40597407.50487186] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=40597407.504872, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=40597407.504872, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

termination on tolfun=1e-11 (Fri Apr 17 01:28:12 2020)
final/bestever f-value = 4.059741e+07 4.059741e+07
incumbent solution: [-4.60450347 -2.13306055 -3.05877837  3.59237957  2.71964777  0.55198861
 -1.23443512 -2.16103699 ...]
std deviations: [2.43473639 2.435003   2.4350849  2.43507987 2.4347861  2.43500805
 2.43511673 2.43515568 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=118717, Fri Apr 17 01:28:12 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 3.834267896377400e+07 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 3.520409554009183e+07 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [42701543.4243772] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [38344436.68814185] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 3.421701418527421e+07 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   31    589 1.521625161937428e+07 1.1e+00 1.97e+00  2e+00  2e+00 0:03.3
   69   1311 3.329325621942438e+06 1.1e+00 1.57e+00  2e+00  2e+00 0:07.4
  100   1900 -7.225772508218179e+06 1.2e+00 1.26e+00  1e+00  1e+00 0:10.8
  156   2964 -1.821402534667081e+07 1.2e+00 8.26e-01  8e-01  8e-01 0:16.9
  200   3800 -2.418885597882801e+07 1.2e+00 5.84e-01  6e-01  6e-01 0:21.6
  274   5206 -2.989822220049703e+07 1.2e+00 2.99e-01  3e-01  3e-01 0:29.7
  300   5700 -3.058098480302035e+07 1.2e+00 2.50e-01  2e-01  3e-01 0:33.3
  373   7087 -3.198908325922803e+07 1.2e+00 1.54e-01  2e-01  2e-01 0:43.4
  400   7600 -3.235411771878434e+07 1.2e+00 1.27e-01  1e-01  1e-01 0:46.3
  500   9500 -3.291410911935817e+07 1.4e+00 8.38e-02  8e-02  9e-02 0:57.1
  600  11400 -3.327963322024752e+07 1.6e+00 7.16e-02  7e-02  8e-02 1:08.0
  700  13300 -3.360198837088858e+07 1.9e+00 6.70e-02  6e-02  7e-02 1:18.8
  800  15200 -3.446720191281529e+07 2.3e+

In [12]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F9_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F9(X_Values.iloc[i,:200])
    SVM_Fun [i] = F9(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F9(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([94775.48999991]), count=array([1])) 0.0021498662362760736
SVM
ModeResult(mode=array([40608050.70966978]), count=array([1])) 6770771.503511773
ELN
ModeResult(mode=array([37446105.00453839]), count=array([1])) 9468233.331096893
